# Chapter 5: Eager and Lazy APIs

In [1]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.0.0

## Eager API: DataFrame

In [3]:
import polars as pl

## Lazy API: LazyFrame

In [5]:
ldf = pl.LazyFrame({
    "name": ["Alice", "Bob", "Charlie"],
    "age": [25, 30, 35]
})

erroneous_query = ldf.with_columns(
    pl.col("age").str.slice(1,3).alias("sliced_age")
)

result_df = erroneous_query.collect()

## Performance Differences

In [7]:
lf = pl.LazyFrame({"col1": [1,2,3], "col2": [4,5,6]})
# Some heavy computation
print(lf.collect())
print(lf.with_columns(pl.col("col1") + 1).collect())  # Recalculates the LazyFrame

## Functionality Differences

### Aggregations

### Attributes

### Computation

### Descriptive

### GroupBy

### Exporting

### Manipulation and Selection

### Miscellaneous

## Out-of-Core Computation with Lazy API's Streaming Mode

## Tips and Tricks

### Going from LazyFrame to DataFrame and Vice Versa

### Joining a DataFrame and a LazyFrame

In [21]:
lf = pl.LazyFrame({"id": [1,2,3], "value1": [4,5,6]})
df = pl.DataFrame({"id": [1,2,3], "value2": [7,8,9]})

lf.join(df, on="id")

In [22]:
lf = pl.LazyFrame({"id": [1,2,3], "value1": [4,5,6]})
df = pl.DataFrame({"id": [1,2,3], "value2": [7,8,9]})

lf.join(df.lazy(), on="id")

### Caching Intermittent Stages

In [24]:
lf = pl.LazyFrame({"col1": [1,2,3], "col2": [4,5,6]})
# Some heavy computation
lf = lf.collect().lazy()
print(lf.collect())
print(lf.with_columns(pl.col("col1") + 1).collect())  # Utilizes the cached LazyFrame

## Conclusion